<a href="https://colab.research.google.com/github/psygrammer/psypy/blob/master/notebooks/03_b_funcions_and_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 내장 자료구조, 함수, 파일 (2)

* 싸이그래머 / PsyPy [1]
* 김무성

In [88]:
!python -V

Python 3.6.8


# 차례 
* 함수
    - 네임스페이스, 스코프, 지역 함수
    - 여러 값 반환하기
    - 함수도 객체다
    - 익명 함수
    - 클로저 : 함수를 반환하는 함수
    - \*args와  \*\*kwargs를 사용해서 호출 문법 확장하기  
    - 커링 : 일부 인자만 취하기
    - 제너레이터
    - 에러와 예외 다루기
* 파일과 운영체제
    - 바이트와 유니코드 파일
* 클래스
* 함수형 프로그래밍
  - map
  - filter
  - reduce

------------------------

# 함수

```python
def my_function(x, y, z=1.5) :
    if z > 1 :
        return z * (x + y)
    else :
        return z / (x + y)
    
# return문이 몇 개가 되든 상관없다. 
# 함수 블록이 끝날 때까지 return문이 없다면 None이 반환된다.   
```

## 네임스페이스, 스코프, 지역 함수

```python
# 로컬
def func() :
    a = []  # <- local
    for i in range(5) :
        a.append(i)
```        

```python
# 전역 
a = []
def func() :
    for i in range(5) :
        a.append(i)
        
func()
a
```

In [89]:
b = None
def bind_b_variable() :
    b = []
    
bind_b_variable()    
print(b)    

None


In [90]:
# global 예약어 사용
b = None
def bind_b_variable() :
    global b
    b = []

bind_b_variable()
print(b)

[]


```python
# 함수 안에서 함수를 선언할 수 있다.
def outer_function(x, y, z) :
    def inner_function(a, b, c) :
        pass
    pass
```

## 여러 값 반환하기

In [91]:
def f() :
    a = 5
    b = 6
    c = 7
    return a, b, c

a, b, c = f()
print(a, b, c)

5 6 7


In [92]:
return_value = f()
return_value # 여래 개의 값을 하나의 변수에 반환받으면, 그 변수는 튜플 자료형이 된다.

(5, 6, 7)

## 함수도 객체다

In [0]:
states = [ '  Alabama ', 'Georgia!', 'Georgia', 'georgia', 'FlOrIda', 
          'south carolin##', 'West virginaia?']

### 문자열을 변형하기 위해 함수를 사용하는 일반적인 패턴

In [0]:
import re 

def clean_strings(strings) :
    result = []
    for value in strings :
        value = value.strip()
        value = re.sub('[!#?]', '', value) # 문장 부호 제거
        value = value.title()
        result.append(value)
    return result

In [95]:
clean_strings(states)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

### 함수를 객체화하고 인자로 넘겨서 좀 더 유연하게 사용하는 패턴

In [0]:
def remove_punctuation(value) :
    return re.sub('[!#?]', '', value)

# 함수도 객체. 
clean_ops = [str.strip, remove_punctuation, str.title]

def clean_strings(strings, ops) :
    result = []
    for value in strings :
        for function in ops :
            value = function(value) 
        result.append(value)
    return result

In [97]:
clean_strings(states, clean_ops)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

In [98]:
# map을 이용해본다. 어떤 컬렉션에 대해 함수를 적용하는 함수다. 
# 이를 이용해서 함수를 인자로 사용할 수 있다.
map(remove_punctuation, states)

In [99]:
for e in map(remove_punctuation, states) :
    print(e)

  Alabama 
Georgia
Georgia
georgia
FlOrIda
south carolin
West virginaia


## 익명 함수 

```python
def short_function(x) :
    return x * 2

# 위의 함수와 동치인 익명 함수(람다 함수)
equiv_anon = lambda x: x * 2
```

In [100]:
# 각 문자열에서 다양한 문자가 포함된 순서로 정렬하는 예제
strings = ['foo', 'card', 'bar', 'aaaa', 'abab']
strings.sort(key=lambda x: len(set(list(x))))
strings

['aaaa', 'foo', 'abab', 'bar', 'card']

## 클로저 : 함수를 반환하는 함수

In [0]:
# 클로저는 다른 함수에서 반환되는, 동적으로 생성된 함수다. 
# 핵심은 반환되는 함수는 그 함수가 생성된 시점의 
# 지역 네임스페이스의 변수에 접근할 수 있다는 점이다.

In [102]:
def make_closure(a) :
    def closure() :
        print("I know the secret: %d", a)
    return closure

closure = make_closure(5)
closure()
closure() # 클로저를 생성하는 함수가 끝나더라도 생성된 시점의 네임스페이스에 여전히 접근할 수 있다.


I know the secret: %d 5
I know the secret: %d 5


In [0]:
# 클로져가 변경 가능한 객체를 가질 수도 있다.
def make_watcher() :
    have_seen = {}
    
    def has_been_seen(x) :
        if x in have_seen :
            return True
        else :
            have_seen[x] = True
            return False
        
    return has_been_seen

In [104]:
watcher = make_watcher()
vals = [5, 6, 1, 5, 1, 6, 3, 5]
[watcher(x) for x in vals]

[False, False, False, True, True, True, False, True]

## \*args와  \*\*kwargs를 사용해서 호출 문법 확장하기  

```python
# 파이썬에서 함수 인자가 동작하는 방식은, 
# 만약 func(a, b, c, d=some, e= value) 라고 코드를 작성하면
# 일반 인자와 키워드 인자는 
# 각각 튜플 args와
# 사전 kwargs에 저장된다.
# 내부적으로 다음과 같은 과정을 거친다
a, b, c = args
d = kwargs.get('d', d_default_value)
e = kwargs.get('e', e_defaulT_value)
```

In [105]:
# 예제
def say_hello_then_call_f(f, *args, **kwargs) :
    print('args is', args)
    print('kwargs is', kwargs)
    print("Hello! Now I'm going to call %s" %(f))
    return f(*args, **kwargs)

def g(x, y, z=1) :
    return (x+y)/z

say_hello_then_call_f(g, 1, 2, z=5.)

args is (1, 2)
kwargs is {'z': 5.0}
Hello! Now I'm going to call <function g at 0x7f9591c27488>


0.6

## 커링 : 일부 인자만 취하기

In [0]:
# 함수에서 일부 인자를 고정해 새로운 함수를 만드는 기법을 커링이라고 한다.

def add_numbers(x, y) :
    return x + y

# 여기서 add_numbers의 두 번째 인자가 '커리되었다'라고 한다.
add_five = lambda y: add_numbers(5, y) 


# 다음 방식으로 더 단순하게 만들 수 있다.
from functools import partial
add_five = partial(add_numbers, 5)

## 제너레이터

* 이터레이터 프로토콜
* 제너레이터
* 제너레이터 표현식
* itertoos 모듈

### 이터레이터 프로토콜

In [107]:
# 파이썬은 리스트 내의 객체나 파일의 각 행 같은 순차적인 자료를 순회하는 일관적인 방법을 제공한다.
# 이터레이터 프로토콜을 이용해서 순회가 가능한 객체를 만들 수 있다.

some_dict = {'a':1, 'b':2, 'c':3}
 
# for key in some_dict 라고 작성하면 
# 파이썬 인터프리터는 some_dict에서 이터레이터를 생성한다.    
for key in some_dict :
    print(key)

a
b
c


In [0]:
dict_iterator = iter(some_dict)

In [109]:
dict_iterator

이터레이터는 for 문과 같은 컨텍스트에서 사용될 경우 객체를 반환한다. 리스트나 리스트와 유사한 객체를 취하는 대부분의 메서드는 순회 가능한 객체도 허용한다. 여기에는 min, max, sum 같은 내장 매서드와 list, tuple 같은 자료구조를 생성하는 메서드도 포함된다. 

In [110]:
list(dict_iterator)

['a', 'b', 'c']

### 제너레이터

In [0]:
# 제너레이터는 순회가 가능한 객체를 생성하는 간단한 방법이다.
# 일반 함수는 실행되면 단일 값을 반환하지만
# 제너레이터는 순차적인 값을 매 요청 시마다 하나씩 반환한다.
# 제너레이터를 생성하려면 함수에서 return 대신에 yeild 예약어를 사용한다.
def squares(n=10) :
    print('Generating squares from 1 to %d' %(n ** 2))
    for i in range(1, n+1) :
        yield i ** 2

In [112]:
gen = squares()
gen

<generator object squares at 0x7f95909c2ba0>

In [113]:
for x in gen :
    print(x) 

Generating squares from 1 to 100
1
4
9
16
25
36
49
64
81
100


### 제너레이터 표현식

In [114]:
# 제너레이터를 만드는 간단한 방법은 제너레이터 표현식을 사용하는 것이다.
gen = (x ** 2 for x in range(100))
gen

<generator object <genexpr> at 0x7f95909c2fc0>

In [115]:
# 위의 코드는 다음 코드와 동일하다
def _make_gen() :
    for x in range(100) :
        yield x ** 2
        
gen = _make_gen()
gen

<generator object _make_gen at 0x7f9591bf9d00>

In [116]:
# 예제
max(x ** 2 for x in range(100))

9801

In [117]:
dict((i, i**2) for i in range(5))

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

### itertools 모듈

In [118]:
# 일반 데이터 알고리즘을 위한 제너레이터를 포함
import itertools

# 예를 들어 groupby는 순차 자료 구조와 함수를 인자로 받아,
# 인자로 받은 함수에서 반환하는 값에 따라 그룹을 지어준다.
first_letter = lambda x : x[0]

names = ['Alan', 'Adam', 'Wes', 'Will', 'Albert', 'Steven']

for letter, names in itertools.groupby(names, first_letter) :
    print(letter, list(names))


A ['Alan', 'Adam']
W ['Wes', 'Will']
A ['Albert']
S ['Steven']


---------------------------------

# 파일과 운영체제 [1, 2]
* 파일 읽기
* with 문 사용
* 파일을 읽는 여러 가지 방법
* 파일 쓰기
* 바이트와 유니코드

------------------------------

실습할 파일을 준비.(웹에서 가져오기 - urllib 사용)

In [119]:
%ls

examples/  sample_data/


In [0]:
# 우선 파일을 가져와보자.

## 1. 디렉토리 만들기
import os

dir_name = "examples"
if not os.path.exists(dir_name) :
  os.mkdir(dir_name)

In [121]:
# example 디렉토리가 생겼다.
%ls

examples/  sample_data/


In [122]:
# 지금은 디렉토리 안에 아무 파일도 없다.
%ls examples/

segismundo.txt  tmp2.txt  tmp.txt


In [0]:
# 2. 파일 가져오기 - [1]의 예제소스 github의 파일들을 주소로 접근해서 가져온다.

import urllib.request

url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/segismundo.txt'
file_name = "examples/segismundo.txt" 
with urllib.request.urlopen(url) as r:
  doc = r.read()
  with open(file_name, "wb") as f:
    f.write(doc)

In [124]:
# 다운로드 완료. 파일이 생겼다.
%ls examples/

segismundo.txt  tmp2.txt  tmp.txt


In [125]:
# 내용를 한번 살펴보자 
!head examples/segismundo.txt

Sueña el rico en su riqueza,
que más cuidados le ofrece;

sueña el pobre que padece
su miseria y su pobreza;

sueña el que a medrar empieza,
sueña el que afana y pretende,
sueña el que agravia y ofende,



--------------

## 파일 읽기

| 파일열기모드	| 설명 
| :------------- | :-------------
|r	| 읽기모드 - 파일을 읽기만 할 때 사용
|w	| 쓰기모드 - 파일에 내용을 쓸 때 사용
|a	| 추가모드 - 파일의 마지막에 새로운 내용을 추가 시킬 때 사용

파일을 읽고 쓰기 위해서는 내장 함수인 open을 이용해서 파일의 상대 경로나 절대 경로를 넘겨주어야 한다.

In [0]:
path = "examples/segismundo.txt"

In [0]:
f = open(path)

기본적으로 파일은 읽기 전용 모드인 'r'로 열린다. 파일 핸들 f를 리스트로 생각할 수 있으며 파일의 매 줄을 순회할 수 있다.

In [128]:
for line in f:
  print(line) # 여기서 줄별로 처리할 코드를 넣으면 된다.

Sueña el rico en su riqueza,

que más cuidados le ofrece;



sueña el pobre que padece

su miseria y su pobreza;



sueña el que a medrar empieza,

sueña el que afana y pretende,

sueña el que agravia y ofende,



y en el mundo, en conclusión,

todos sueñan lo que son,

aunque ninguno lo entiende.





파일에서 읽은 줄은 줄끝(end-of-line, EOL) 문자가 그대로 남아 있으므로 파일에서 읽은 줄에서 이를 제거하는 다음과 같은 코드를 많이 사용한다.(string의 rstrip() 혹은 strip() 코드를 적용)

In [0]:
lines = [x.rstrip() for x in open(path)]

In [130]:
lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

파일 객체를 생성하기 위해 open을 사용했다면 작업이 끝났을 때 명시적으로 닫아주어야 한다. 파일을 닫으면 해당 자원을 운영체제로 되돌려준다.

In [0]:
f.close()

In [0]:
# 위의 코드들을 한번에 쓰면 다음과 같다. 
path = "examples/segismundo.txt"
f = open(path)
for line in f:
  lines = [x.strip() for x in open(path)] # 오른쪽 끝만 정리하는 rstrip 대신에 줄의 양쪽 끝을 다 정리하는 strip 함수 사용

In [133]:
lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

## With 문 사용

with 문을 사용하면 파일 작업이 끝날을 때 필요한 작업을 쉽게 처리할 수 있다.

In [0]:
# 위의 코드와 동일한 기능. with 문 사용.
# 이렇게 하면 with 블록이 끝나는 시점에 파일 핸들 f를 자동으로 닫아준다.  
with open(path) as f:
  lines = [x.strip() for x in f]


In [135]:
lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

## 파일을 읽는 여러 가지 방법
* readline
* readlines
* read
* seek
* tell

### readline

In [136]:
# 줄 단위로 가져옴. 현재 코드는 한줄만.
with open(path) as f:
  line = f.readline()
  print(line)

Sueña el rico en su riqueza,



In [137]:
# 전체 다 가져오려면
with open(path) as f:
  while True:
    line = f.readline()
    if not line: break
    print(line)

Sueña el rico en su riqueza,

que más cuidados le ofrece;



sueña el pobre que padece

su miseria y su pobreza;



sueña el que a medrar empieza,

sueña el que afana y pretende,

sueña el que agravia y ofende,



y en el mundo, en conclusión,

todos sueñan lo que son,

aunque ninguno lo entiende.





### readlines

In [138]:
# 줄 단위로 가져오는데, 한번에 다 읽기
with open(path) as f:
  lines = f.readlines()
  
print(lines)

['Sueña el rico en su riqueza,\n', 'que más cuidados le ofrece;\n', '\n', 'sueña el pobre que padece\n', 'su miseria y su pobreza;\n', '\n', 'sueña el que a medrar empieza,\n', 'sueña el que afana y pretende,\n', 'sueña el que agravia y ofende,\n', '\n', 'y en el mundo, en conclusión,\n', 'todos sueñan lo que son,\n', 'aunque ninguno lo entiende.\n', '\n']


### read

In [139]:
# 파일 전체를 한번에 다 읽어서, 하나의 문자열로 반환.
with open(path) as f:
  doc = f.read()
  
print(doc)

Sueña el rico en su riqueza,
que más cuidados le ofrece;

sueña el pobre que padece
su miseria y su pobreza;

sueña el que a medrar empieza,
sueña el que afana y pretende,
sueña el que agravia y ofende,

y en el mundo, en conclusión,
todos sueñan lo que son,
aunque ninguno lo entiende.




In [140]:
# 특정 개수만큼의 문자까지만 읽을 수도 있다.
# '문자'는 인코딩(UTF-8)으로 결정되거나 이진 모드일 경우 단순히 바이트로 결정된다.
with open(path) as f:
  s = f.read(10)
  
print(s)

Sueña el r


In [141]:
with open(path, 'rb') as f: # 이진 모드
  s = f.read(10)
  
print(s)

b'Sue\xc3\xb1a el '


### tell

In [142]:
# read 메서드는 읽은 바이트만큼 파일 핸들의 위치를 옮긴다. 
# tell 메서드는 현재 위치를 알려준다.
with open(path) as f:
  s = f.read(10)
  print(f.tell())
  
print(s)

11
Sueña el r


위의 경우 파일에서 10개의 문자를 읽었어도 위치가 11인 이유는 기본 인코딩에서 10개의 문자를 디코딩하기 위해 그 만큼의 바이트가 필요했기 때문이다. 

In [143]:
# 이진 모드에서는 정확히 나온다.
with open(path, 'rb') as f: # 이진 모드
  s = f.read(10)
  print(f.tell())
  
print(s)

10
b'Sue\xc3\xb1a el '


시스템의 기본 인코딩은 sys 모듈에서 확인할 수 있다.

In [144]:
import sys

sys.getdefaultencoding()

'utf-8'

### seek

In [145]:
# seek 메서드는 파일 핸들의 위치를 해당 파일에서 지정한 바이트 위치로 옮긴다.
with open(path) as f: 
  f.seek(3)
  s = f.read(10)
  
print(s)

ña el rico


## 파일 쓰기

파일에 텍스트를 기록하려면 'w' 옵션으로 쓰기모드로 파일을 만들고, write나 writenlines 메서드를 이용하면 된다.

In [0]:
# 쓸 파일 이름
dst_path = "examples/tmp.txt"

In [147]:
# 아직은 파일이 생성되지 않았다. 
%ls examples/

segismundo.txt  tmp2.txt  tmp.txt


In [0]:
# 빈 파일 생성
with open(dst_path, 'w') as f:
  pass # 아무것도 하지 않는다.

In [149]:
# 새 파일이 생성되지 않았다. 
%ls examples/

segismundo.txt  tmp2.txt  tmp.txt


In [0]:
# 파일 안에는 아무것도 쓰여지지 않았다.
%cat examples/tmp.txt

In [0]:
with open(dst_path, 'w') as f:
  f.write("Hello world!") # write 메서드는 줄 뛰우기('\n')는 해주지 않는다
  f.write("Hello Python!\n") # 줄 뛰우기('\n')를 직접 해줘야 한다.
  f.write("Hello PsyPy!")

In [152]:
%cat examples/tmp.txt

Hello world!Hello Python!
Hello PsyPy!

In [0]:
# 'w' 모드로 열면 원래 파일의 내용을 덮어써버린다. 
with open(dst_path, 'w') as f:
  f.write("Hi PsyPy~")

In [154]:
%cat examples/tmp.txt

Hi PsyPy~

In [0]:
# 'a' 모드로 열면 원래 파일의 내용에 이어서 쓴다. 
with open(dst_path, 'a') as f:
  f.write("!!!")

In [156]:
%cat examples/tmp.txt

Hi PsyPy~!!!

In [0]:
# 'a' 모드로 열었는데 해당 이름의 파일이 없으면 새로 만들어서 쓴다.
with open("examples/tmp2.txt", 'a') as f:
  f.write("!!!")

In [158]:
%ls examples

segismundo.txt  tmp2.txt  tmp.txt


In [159]:
%cat examples/tmp2.txt

!!!!!!

In [0]:
# 모든 줄을 한번에 다 쓰려면 writelines 메서드를 써보자.
# 다음의 내용은 원래 파일에서 빈 줄이 포함되지 않도록 제거하고 새 파일로 쓰는 예제이다.

with open(dst_path, 'w') as f :
  f.writelines(x for x in open(path) if len(x) > 1)

In [161]:
# 빈 줄들을 제거하고 쓴 새 파일 내용
%cat examples/tmp.txt

Sueña el rico en su riqueza,
que más cuidados le ofrece;
sueña el pobre que padece
su miseria y su pobreza;
sueña el que a medrar empieza,
sueña el que afana y pretende,
sueña el que agravia y ofende,
y en el mundo, en conclusión,
todos sueñan lo que son,
aunque ninguno lo entiende.


In [162]:
# 원본 파일 내용
%cat examples/segismundo.txt

Sueña el rico en su riqueza,
que más cuidados le ofrece;

sueña el pobre que padece
su miseria y su pobreza;

sueña el que a medrar empieza,
sueña el que afana y pretende,
sueña el que agravia y ofende,

y en el mundo, en conclusión,
todos sueñan lo que son,
aunque ninguno lo entiende.



-------------------------------

## Google Drive text와 연결.

참고 문서 링크: https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/

In [208]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


URL로 가서 authorization code를 아래 code 입력란에 copy and paste. 마운트 위치는 /content/gdrive/My Drive/ 이렇게 됨.

In [0]:
!ls '/content/gdrive/My Drive/' # 유닉스 명령어 ls를 사용하여 폴더에 있는 파일 리스트를 확인 가능

자신의 구글 드라이브에 PsyPy 디렉토리를 만들어서 거기에 다음의 파일을 올린다음
* https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/segismundo.txt


In [212]:
# 제대로 만들어서 업로드했으면 다음처럼 확인할 수 있다.
!ls '/content/gdrive/My Drive/PsyPy'

segismundo.txt


In [0]:
path = "/content/gdrive/My Drive/PsyPy/segismundo.txt"

파일을 읽어서, 내용들을 모두 대문자로 바꿔서 새 파일을 만들어 저장(구글 드라이브의 Psypy 디렉토리 안에)해보자.

In [0]:
# insert your code here

------------------------------

실습할 파일을 준비.(github 저장소를 클로닝하기)

In [187]:
# 실습을 위한 소스와 데이터가 있는 저장소를 클로닝해서 colab과 동기화하는 방법을 사용.
# 지금은 클로닝되어 있지 않음
%ls

examples/  sample_data/


In [188]:
# 저장소 가져옴
!git clone https://github.com/psygrammer/psypy.git

Cloning into 'psypy'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 124 (delta 40), reused 81 (delta 13), pack-reused 0
Receiving objects: 100% (124/124), 18.79 MiB | 26.29 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [189]:
# 코드&데이터 디렉토리 확인
%ls

examples/  psypy/  sample_data/


In [192]:
#colab의 실행 위치를 psypy 디렉토리 내부로 옮기자
%cd psypy

/content/psypy


In [193]:
%ls

ml/  notebooks/  README.md


In [194]:
#실행 위치를 notebooks 내부로 한번 더 들어가자.
%cd notebooks

/content/psypy/notebooks


In [195]:
%ls

02_python_basics.ipynb               03_b_funcions_and_files.ipynb  figures/
03_a_built_in_data_structures.ipynb  examples/


In [196]:
# 실습을 위한 파일들 확인
%ls examples/

iu/  person.csv


----------------------------

## 응용 문제 1 - 파일 분석
* examples/person.csv 파일을 파이썬으로 분석한다.

In [197]:
%cat examples/person.csv

no,id,gender,email,com,score
1,ma,male,,naver,100
2,fa,female,,google,100
3,mb,male,,yahoo,80
4,mc,male,,daum,70
5,fb,female,,daum,60
6,fc,female,,naver,70
7,me,male,,yahoo,80
8,md,male,,naver,100
9,mg,male,,naver,90
10,fd,female,,naver,90

In [0]:
path = "examples/person.csv"

### 1) score의 평균을 구하기

In [0]:
# insert your code here

### 2) 남,녀별 score 평균 구하기

In [0]:
# insert your code here

### 3) 각 email의 값은 비워져있다. 새 파일에 채워넣어보자. 이메일은 '아이디@회사이름.com' 규칙을 따른다. 예를들어 id가 user1이고 회사가 naver이면 그 사람의 email은 user1@naver.com 이다.¶ 

In [0]:
# insert your code here

## 응용 문제 2 - 디렉토리의 사진 정리
* examples/iu 디렉토리에는 iu 사진이 들어있다.
* 디렉토리 안에 하위 디렉토리를 생성해서 사진 파일 포맷별로 정리해보자.
* 예를들어 jpg 파일들은 jpg 파일별로 모아서 새로 디렉토리를 생성해서 그곳에 넣는다.
예) iu/1.jpg, 2.jpg, 3.jpg --> iu/jpg/1.jpg, 2.jpg, 3.jpg

In [0]:
# insert your code here

## 응용문제 3 - 간단한 웹 크롤링
* 다음 주소로 방탄소년단의 노래 가사를 가져온다.
* 가사 중에 영어가 섞인 문장은 제거하고 출력해보자. 

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
#url = "https://gasazip.com/view.html?no=614736"
#url = "https://gasazip.com/view.html?no=636135"
url = "http://gasazip.com/view.html?no=2276458"

In [0]:
# HTTP GET Request
req = requests.get(url)
# HTML 소스 가져오기
html = req.text
# BeautifulSoup으로 html소스를 python객체로 변환하기
# 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시.
# 여기서는 Python 내장 html.parser를 이용했다.
soup = BeautifulSoup(html, 'html.parser')

In [0]:
lyrics = []
for txt in soup.find_all('div', attrs={'class': 'col-md-8'}) :
    lines = txt.get_text().split('\n')
    for line in lines :
        lyrics.append(line.strip())

In [207]:
lyrics

['작은 것들을 위한 시 (Boy With Luv)Feat.Halsey',
 '',
 '모든 게 궁금해',
 'How’s your day',
 'Oh tell me',
 '뭐가 널 행복하게 하는지',
 'Oh text me',
 'Your every picture',
 '내 머리맡에 두고 싶어',
 'oh bae',
 'Come be my teacher',
 '네 모든 걸 다 가르쳐줘',
 'Your 1 your 2',
 'Listen my my baby 나는',
 '저 하늘을 높이 날고 있어',
 '그때 니가 내게 줬던 두 날개로',
 '이제 여긴 너무 높아',
 '난 내 눈에 널 맞추고 싶어',
 'Yeah you makin’ me a boy with luv',
 'Oh my my my oh my my my',
 "I've waited all my life",
 '네 전부를 함께하고 싶어',
 'Oh my my my oh my my my',
 'Looking for something right',
 '이제 조금은 나 알겠어',
 'I want something stronger',
 'Than a moment',
 'than a moment love',
 'I have waited longer',
 'For a boy with',
 'For a boy with luv',
 '널 알게 된 이후 ya',
 '내 삶은 온통 너 ya',
 '사소한 게 사소하지 않게',
 '만들어버린 너라는 별',
 '하나부터 열까지 모든 게 특별하지',
 '너의 관심사 걸음걸이 말투와',
 '사소한 작은 습관들까지',
 '다 말하지 너무 작던',
 '내가 영웅이 된 거라고',
 'Oh nah',
 '난 말하지 운명 따윈',
 '처음부터 내 게 아니었다고',
 'Oh nah',
 '세계의 평화',
 'No way',
 '거대한 질서',
 'No way',
 '그저 널 지킬 거야 난',
 'Boy with luv',
 'Listen my my baby 나는',
 '저 하늘을 높이 날고

In [0]:
# insert your code here

------------------------

# 클래스 [3, 4]

In [0]:
# 클래스를 하나 만들기 위해 특정 객체의 하위 클래스를 만들 수 있습니다.
class Human(object):

    # 클래스 속성은 이 클래스의 모든 인스턴스에서 공유합니다.
    species = "H. sapiens"

    # 기본 초기화자
    def __init__(self, name):
        # 인자를 인스턴스의 name 속성에 할당합니다.
        self.name = name

    # 모든 인스턴스 메서드에서는 self를 첫 번째 인자로 받습니다.
    def say(self, msg):
       return "%s: %s" % (self.name, msg)

    # 클래스 메서드는 모든 인스턴스에서 공유합니다.
    # 클래스 메서드는 호출하는 클래스를 첫 번째 인자로 호출됩니다.
    @classmethod
    def get_species(cls):
        return cls.species

    # 정적 메서드는 클래스나 인스턴스 참조 없이도 호출할 수 있습니다.
    @staticmethod
    def grunt():
        return "*grunt*"

In [164]:
# 클래스 인스턴스화
i = Human(name="Ian")
print(i.say("hi"))     

Ian: hi


In [165]:
j = Human("Joel")
print(j.say("hello"))

Joel: hello


In [166]:
# 클래스 메서드를 호출
i.get_species() 

'H. sapiens'

In [167]:
# 공유 속성을 변경
Human.species = "H. neanderthalensis"
print(i.get_species()) 
print(j.get_species()) 

H. neanderthalensis
H. neanderthalensis


In [168]:
# 정적 메서드를 호출
Human.grunt()

'*grunt*'

In [0]:
class HousePark:
    lastname = "박"
    def __init__(self, name):
        self.fullname = self.lastname + name
    def travel(self, where):
        print("%s, %s여행을 가다." % (self.fullname, where))
    def love(self, other):
        print("%s, %s 사랑에 빠졌네" % (self.fullname, other.fullname))
    def fight(self, other):
        print("%s, %s 싸우네" % (self.fullname, other.fullname))
    def __add__(self, other):
        print("%s, %s 결혼했네" % (self.fullname, other.fullname))
    def __sub__(self, other):
        print("%s, %s 이혼했네" % (self.fullname, other.fullname))
    def __del__(self):
        print("%s 죽네" % self.fullname)

class HouseKim(HousePark):
    lastname = "김"
    def travel(self, where, day):
        print("%s, %s여행 %d일 가네." % (self.fullname, where, day))

In [0]:
pey = HousePark("응용")
juliet = HouseKim("줄리엣")

In [171]:
pey.travel("부산")
juliet.travel("부산", 3)
pey.love(juliet)
pey + juliet
pey.fight(juliet)
pey - juliet

박응용, 부산여행을 가다.
김줄리엣, 부산여행 3일 가네.
박응용, 김줄리엣 사랑에 빠졌네
박응용, 김줄리엣 결혼했네
박응용, 김줄리엣 싸우네
박응용, 김줄리엣 이혼했네


In [172]:
pey.__del__()

박응용 죽네


--------------------

# 함수형 프로그래밍 [5]

* map
* filter
* reduce

## map

map이라는 것은 함수와 시퀀스 자료형(리스트, 터플, 문자열)을 입력으로 받아서 시퀀스 자료형의 각각의 요소가 함수의 입력으로 들어간 다음 나오는 출력값을 묶어서 리스트로 돌려주는 함수이다.

In [173]:
def f(x) :
    return x*x

x = [1, 2, 3, 4, 5]
y = map(f, x)

print(x)
print(list(y))

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.7.png?raw=1'/>

In [174]:
# 위의 코드는 다음과 같다
y = []
for e in x :
    ne = f(e)
    y.append(ne)

print(x)
print(y)

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


In [175]:
# 람다 함수 이용
y = map(lambda a:a*a, x)
print(x)
print(list(y))

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


In [176]:
x = range(10)
y = map(lambda a: a*a+4*a, x)
print(x)
print(list(y))

range(0, 10)
[0, 5, 12, 21, 32, 45, 60, 77, 96, 117]


In [177]:
# 두 개 이상 입력
x = [1,2,3,4,5]
y = [6,7,8,9,10]
z = map(lambda a, b:a+b, x, y)
print(x)
print(y)
print(list(z))

[1, 2, 3, 4, 5]
[6, 7, 8, 9, 10]
[7, 9, 11, 13, 15]


<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.8.png?raw=1'/>

## filter

filter는 함수와 시퀀스 자료형을 입력으로 받아서 자료의 값이 하나씩 함수의 인수로 전달될 때, 참을 반환시키는 값만을 따로 모아서 리스트의 형태로 반환하는 함수이다. filter의 뜻은 무엇인가를 걸러낸다는 뜻이다. 이 의미가 filter 함수에서도 그대로 사용된다.

In [178]:
def positive(x): 
    return x > 0

list(filter(positive, [1,-3,2,0,-5,6]))

[1, 2, 6]

In [179]:
# 위의 코드는 다음과 같다
def positive(l): 
    result = [] 
    for i in l: 
        if i > 0: 
            result.append(i) 
    return result

positive([1,-3,2,0,-5,6])

[1, 2, 6]

In [180]:
# 람다 함수 사용
list(filter(lambda x : x>0, [1,-3,2,0,-5,6]))

[1, 2, 6]

In [181]:
list(filter(lambda x : x%2, [1,2,3,4,5,6]))

[1, 3, 5]

<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.11.png?raw=1'/>

## reduce

reduce는 첫 인수로 함수를 받는다. 이 함수는 두 개의 인수를 받는다. 두 번째 인자는 시퀀스 자료형이다. 시퀀스의 자료들은 순차적으로 reduce가 받은 함수의 첫 번째 인수, 두 번째 인수로 전달된다. 첫 인수는 함수의 계산 결과가 누적적으로 적용된다. 단, 처음 계산에서는 시퀀스의 두 개의 요소가 함수에 전달된다.

In [0]:
from functools import reduce

In [183]:
# 1부터 5까지 더하기
# 다음 코드는 이 수식과 동일하다. ((((1+2)+3)+4)+5)
reduce(lambda x, y : x+y, [1, 2, 3, 4, 5])

15

<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.12.png?raw=1'/>

In [184]:
# reduce 함수에 세 번째 인수를 부여할 수 있는데, 초기 값으로 사용된다.
# 다음 코드는 이 수식과 동일하다. (((((0+1)+2)+3)+4)+5)
reduce(lambda x, y : x+y, [1, 2, 3, 4, 5], 0)

15

In [185]:
# 각 원소를 제곱해서 모두 더하기
reduce(lambda x, y : x + y*y, range(1,11), 0)

385

In [186]:
# 위의 코드는 다음과 동일하다.
x = 0
for y in range(1, 11) :
    x = x + y*y
    
x

385

-------------------------------

# 참고자료 
* [1] 파이썬 라이브러리를 활용한 데이터 분석(2판)
    - http://www.hanbit.co.kr/store/books/look.php?p_code=B6417848794
* [2] 점프 투 파이썬 : 04장 프로그램의 입력과 출력은 어떻게 해야 할까? - https://wikidocs.net/23
* [3] python | learn x in y minutes - https://learnxinyminutes.com/docs/python3/
* [4] 점프 투 파이썬 : 클래스 - https://wikidocs.net/28
* [5] 점프 투 파이썬 : 내장함수 - https://wikidocs.net/32